# Theano tutorial
Found on http://deeplearning.net/software/theano/tutorial/index.html

# Import

In [31]:
import theano
import theano.tensor as T
from theano import function
from theano import pp
import numpy as np
from theano import In

# Simple algebra

In [15]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)

print pp(z)
f(2, 3)

(x + y)


array(5.0)

In [18]:
A = T.dmatrix('A')
B = T.dmatrix('B')
C = A + B
matrix_add = function([A, B], C)

X = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])
Y = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])
matrix_add(X, Y)

array([[  2.,   4.,   6.],
       [  4.,   6.,   8.],
       [  6.,   8.,  10.]])

In [22]:
a = T.dvector('a')
b = T.dvector('b')
c = a**2 + b**2 + 2*a*b

quad_func = function([a, b], c)
quad_func(np.array([1, 2]), np.array([1, 2]))

array([  4.,  16.])

# Examples of use
## Logistic function

The formula is:

$$ s = \frac{1}{1 + e^{-x}}$$

or

$$ s = \frac{1 + tanh(x/2)}{2} $$

In [25]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))

logistic = function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [29]:
s2 = (1 + T.tanh(x / 2)) / 2
logistic2 = function([x], s2)

a = np.array([[0, 1], [-1, -2]])
np.allclose(logistic(a), logistic2(a))

True

## Multiple outputs

In [30]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2

multiple_out = function([a, b], [diff, abs_diff, diff_squared])
multiple_out([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

## Default value

In [38]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([In(x, value=1), In(y, value=1, name='y_name')], z)
f(32, y_name=3)

array(35.0)

## Shared state

In [41]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

print state.get_value()
accumulator(1)
print state.get_value()
accumulator(20)
print state.get_value()
state.set_value(0)
print state.get_value()

0
1
21
0


In [44]:
decrementor = function([inc], state, updates=[(state, state-inc)])
decrementor(5)
print state.get_value()

-10


In [48]:
fn_of_state = state * 3 + inc
new_state = T.scalar(dtype=state.dtype)
skip_shared = function([inc, new_state], fn_of_state, givens=[(state, new_state)])
print skip_shared(1, 3)
print state.get_value()

10
-10


## Copying functions

In [65]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)], 
                              on_unused_input='ignore')

In [54]:
print accumulator(10)
print state.get_value()

30
40


In [59]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state:new_state})
print new_accumulator(100)
print new_state.get_value()
print state.get_value()

[array(0)]
100
40


In [66]:
null_accumulator = accumulator.copy(delete_updates=True)

In [67]:
print null_accumulator(9000)
print state.get_value()

[array(0)]
0


## Random numbers

In [68]:
from theano.tensor.shared_randomstreams import RandomStreams

In [69]:
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2, 2))
rv_n = srng.normal((2, 2))
f = function([], rv_u)
g = function([], rv_n)
nearly_zeros = function([], rv_u + rv_n - 2 * rv_u)

In [73]:
print f()
print g()

[[ 0.86342685  0.81031029]
 [ 0.86695784  0.6813093 ]]
[[ 1.15903744 -0.81196202]
 [ 0.14753671  0.14376777]]


In [75]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(1100)
rv_u.rng.set_value(rng_val, borrow=True)
print srng.seed(1200)

None
